#### Training

In [2]:
import time
import pickle
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from copy import deepcopy
from dhg import Hypergraph
from dhg.models import HGNN
from dhg.metrics import HypergraphVertexClassificationEvaluator as Evaluator

In [3]:
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [4]:
def train(net, X, G, y, train_idx, optimizer, epoch):
    net.train()
    start = time.time()
    optimizer.zero_grad()
    out = net(X, G)
    loss = F.cross_entropy(out[train_idx], y[train_idx])
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch}, Time: {time.time()-start:.4f}s, Loss: {loss.item():.4f}")
    return loss.item()

In [4]:
with open("/home/mhoque2/akidul_projects/FLASH/Flash-IDS/testing/HGNN_training/cadets_train_knn_10hop.pkl", "rb") as f:
        edge_list = pickle.load(f)
print("Max hyperedge size:", max(len(edge) for edge in edge_list))

Max hyperedge size: 4


#### Extra Attention

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def segment_sum(src, index, num_segments):
    out = torch.zeros((num_segments,) + src.shape[1:], dtype=src.dtype, device=src.device)
    return out.index_add_(0, index, src)

def segment_max(src, index, num_segments):
    # initialize with -inf
    out = torch.full((num_segments,) + src.shape[1:], -float("inf"), dtype=src.dtype, device=src.device)
    # scatter max via reduce: loopless with scatter_reduce if available, else manual
    # PyTorch >=2.0: torch.scatter_reduce_ (if available)
    try:
        out2 = out.clone()
        out2.scatter_reduce_(0, index.view(-1, *([1]*(src.dim()-1))).expand_as(src), src, reduce="amax", include_self=True)
        return out2
    except Exception:
        # Fallback: do a safe max using buckets (works well enough)
        out = out.clone()
        for i in range(num_segments):
            mask = (index == i)
            if mask.any():
                out[i] = torch.max(src[mask], dim=0).values
        return out

def segment_softmax(scores, index, num_segments):
    # softmax per segment i: exp(s - max_i)/sum_i exp(...)
    max_per_seg = segment_max(scores.unsqueeze(-1), index, num_segments).squeeze(-1)
    s_centered = scores - max_per_seg[index]
    exp_s = torch.exp(s_centered)
    denom = segment_sum(exp_s.unsqueeze(-1), index, num_segments).squeeze(-1)
    return exp_s / (denom[index] + 1e-12)


In [7]:
class _HyperedgeAttentionPool_pt(nn.Module):
    def __init__(self, in_dim, attn_dim=None, topk=None):
        super().__init__()
        a = attn_dim or in_dim
        self.key = nn.Linear(in_dim, a, bias=False)
        self.val = nn.Linear(in_dim, in_dim, bias=False)
        self.query = nn.Parameter(torch.randn(a))
        self.topk = topk

    def forward(self, x, V2E, num_edges):
        v_ids, e_ids = V2E[0], V2E[1]  # [E_nnz]
        K = self.key(x)                # [N, a]
        V = self.val(x)                # [N, d]
        scores_full = (K[v_ids] @ self.query)  # [E_nnz]

        if self.topk is not None:
            # simple top-k per edge using Python buckets (OK on CPU; set topk=None if very big)
            from collections import defaultdict
            keep = torch.zeros_like(scores_full, dtype=torch.bool)
            buckets = defaultdict(list)
            for i, e in enumerate(e_ids.tolist()):
                buckets[e].append(i)
            for e, idxs in buckets.items():
                k = min(self.topk, len(idxs))
                s = scores_full[idxs]
                top_idx = torch.topk(s, k=k).indices
                for j in top_idx.tolist():
                    keep[idxs[j]] = True
            scores = scores_full.masked_fill(~keep, float("-inf"))
        else:
            scores = scores_full

        alpha = segment_softmax(scores, e_ids, num_edges)       # [E_nnz]
        h_e = segment_sum(V[v_ids] * alpha.unsqueeze(-1), e_ids, num_edges)  # [M, d]
        return h_e, alpha

class AttnHGNN_DHG_pt(nn.Module):
    def __init__(self, in_dim, hid, num_classes, edge_list, dropout=0.2, attn_dim=None, topk=None):
        super().__init__()
        # ---- build incidence directly from the provided edge_list
        # edge_list: List[List[int]] or List[Tuple[int,...]]
        edge_list = [list(map(int, e)) for e in edge_list]

        node_ids, edge_ids = [], []
        for e_id, nodes in enumerate(edge_list):
            node_ids.extend(nodes)
            edge_ids.extend([e_id]*len(nodes))

        V2E = torch.tensor([node_ids, edge_ids], dtype=torch.long)
        E2V = torch.stack((V2E[1], V2E[0]), dim=0)
        M = len(edge_list)

        # edge degrees for spread
        e_ids = E2V[0]
        edge_deg = segment_sum(torch.ones_like(e_ids, dtype=torch.float32), e_ids, M).clamp(min=1)

        # register buffers so they move with .to(device)
        self.register_buffer("V2E", V2E)
        self.register_buffer("E2V", E2V)
        self.register_buffer("edge_deg", edge_deg)
        self.M = M

        self.pool1 = _HyperedgeAttentionPool_pt(in_dim, attn_dim, topk)
        self.lin1  = nn.Linear(in_dim, hid, bias=False)
        self.pool2 = _HyperedgeAttentionPool_pt(hid, attn_dim, topk)
        self.lin2  = nn.Linear(hid, hid, bias=False)
        self.cls   = nn.Linear(hid, num_classes)
        self.act   = nn.ReLU()
        self.drop  = nn.Dropout(dropout)
        self._last_alpha = None

    def edge_to_node(self, h_e, num_nodes):
        e_ids, v_ids = self.E2V[0], self.E2V[1]
        contrib = h_e[e_ids] / self.edge_deg[e_ids].unsqueeze(-1)
        return segment_sum(contrib, v_ids, num_nodes)

    def forward(self, X, G):
        N = X.size(0)
        h_e1, a1 = self.pool1(X, self.V2E, self.M)
        X1 = self.edge_to_node(h_e1, N)
        X1 = self.drop(self.act(self.lin1(X1)))

        h_e2, a2 = self.pool2(X1, self.V2E, self.M)
        X2 = self.edge_to_node(h_e2, N)
        X2 = self.drop(self.act(self.lin2(X2)))

        self._last_alpha = (a1, a2)
        return self.cls(X2)


In [8]:
# OLD (that errored)
# net = AttnHGNN_DHG_pt(..., G=G, ...)

# NEW: pass the same edge_list you loaded from pkl
net = AttnHGNN_DHG_pt(
    in_dim=X.shape[1],
    hid=64,
    num_classes=int(torch.unique(y).numel()),
    edge_list=edge_list,      # <— pass edge_list directly
    dropout=0.2,
    attn_dim=None,
    topk=None                 # set e.g. 128 if edges can be very large
).to(torch.device("cpu"))


NameError: name 'X' is not defined

In [55]:
print("Number of hyperedges:", len(edge_list))

Number of hyperedges: 362602


In [9]:
if __name__ == "__main__":
    # --- Setup
    set_seed(2023)
    device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
    #device = torch.device("cpu")

    print(device)

    # --- Load your data
    X = torch.tensor(np.load("/home/mhoque2/akidul_projects/FLASH/Flash-IDS/testing/HGNN_training/cadets_features.npy"), dtype=torch.float)
    y = torch.tensor(np.load("/home/mhoque2/akidul_projects/FLASH/Flash-IDS/testing/HGNN_training/cadets_labels.npy"), dtype=torch.long)

    num_nodes = X.shape[0]
    G = Hypergraph(num_nodes, edge_list)

    # --- Use all indices for training
    train_idx = torch.arange(len(y), dtype=torch.long)

    # --- Move to GPU
    X, y = X.to(device), y.to(device)
    G = G.to(device)

    # --- Define Model
    #net = HGNN(X.shape[1], 64, torch.unique(y).numel(), use_bn=True).to(device)
    net = AttnHGNN_DHG_pt(
    in_dim=X.shape[1],
    hid=64,
    num_classes=int(torch.unique(y).numel()),
    edge_list=edge_list,   # <-- pass edge_list, not G
    dropout=0.2,
    attn_dim=None,
    topk=None
).to(device)
    optimizer = optim.Adam(net.parameters(), lr=0.01, weight_decay=5e-4)

    # --- Train
    for epoch in range(50):
        train(net, X, G, y, train_idx, optimizer, epoch)

    # --- Optional: Save model
    torch.save(net.state_dict(), "knn_10_hop_train_cadet_sep_2.pth")
    # torch.save(net.state_dict(), "contrast_cadets_full_training_pair.pth")
    #torch.save(net.state_dict(), "hgnn_cadets_full_training_HDBSCAN.pth")
    print("\nTraining complete. Model saved as 'hgnn_cadets_full_training.pth'")


cuda:3
Epoch 0, Time: 1.1969s, Loss: 6686.7256
Epoch 1, Time: 0.0358s, Loss: 2.2500
Epoch 2, Time: 0.0366s, Loss: 51.6000
Epoch 3, Time: 0.0510s, Loss: 3.0745
Epoch 4, Time: 0.0350s, Loss: 12.0310
Epoch 5, Time: 0.0418s, Loss: 2.2915
Epoch 6, Time: 0.0361s, Loss: 1.5634
Epoch 7, Time: 0.0305s, Loss: 1.6510
Epoch 8, Time: 0.0341s, Loss: 20.1193
Epoch 9, Time: 0.0343s, Loss: 44.8789
Epoch 10, Time: 0.0313s, Loss: 2.6398
Epoch 11, Time: 0.0303s, Loss: 6.1614
Epoch 12, Time: 0.0287s, Loss: 41.4561
Epoch 13, Time: 0.0336s, Loss: 23.2552
Epoch 14, Time: 0.0298s, Loss: 42.0183
Epoch 15, Time: 0.0711s, Loss: 1.6584
Epoch 16, Time: 0.0247s, Loss: 1.5142
Epoch 17, Time: 0.0345s, Loss: 1.4677
Epoch 18, Time: 0.0334s, Loss: 1.4213
Epoch 19, Time: 0.0306s, Loss: 1.3924
Epoch 20, Time: 0.0555s, Loss: 1.3765
Epoch 21, Time: 0.0328s, Loss: 1.3472
Epoch 22, Time: 0.0326s, Loss: 1.3342
Epoch 23, Time: 0.0376s, Loss: 1.3271
Epoch 24, Time: 0.0303s, Loss: 1.3071
Epoch 25, Time: 0.0298s, Loss: 1.2905
Epoch

#### regular knn 10 hop and combined json evaluattion both

In [9]:
import time
import pickle
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from copy import deepcopy
from dhg import Hypergraph
from dhg.models import HGNN
from dhg.metrics import HypergraphVertexClassificationEvaluator as Evaluator

In [10]:
def add_node_properties(nodes, node_id, properties):
    if node_id not in nodes:
        nodes[node_id] = []
    nodes[node_id].extend(properties)

def update_edge_index(edges, edge_index, index):
    for src_id, dst_id in edges:
        src = index[src_id]
        dst = index[dst_id]
        edge_index[0].append(src)
        edge_index[1].append(dst)

def prepare_graph(df):
    nodes, labels, edges = {}, {}, []
    dummies = {'SUBJECT_PROCESS': 0, 'FILE_OBJECT_FILE': 1, 'FILE_OBJECT_UNIX_SOCKET': 2, 
               'UnnamedPipeObject': 3, 'NetFlowObject': 4, 'FILE_OBJECT_DIR': 5}
    
    for _, row in df.iterrows():
        action = row["action"]
        properties = [row['exec'], action] + ([row['path']] if row['path'] else [])
        
        actor_id = row["actorID"]
        add_node_properties(nodes, actor_id, properties)
        labels[actor_id] = dummies[row['actor_type']]

        object_id = row["objectID"]
        add_node_properties(nodes, object_id, properties)
        labels[object_id] = dummies[row['object']]

        edges.append((actor_id, object_id))

    features, feat_labels, edge_index, index_map = [], [], [[], []], {}
    for node_id, props in nodes.items():
        features.append(props)
        feat_labels.append(labels[node_id])
        index_map[node_id] = len(features) - 1

    update_edge_index(edges, edge_index, index_map)

    return features, feat_labels, edge_index, list(index_map.keys())

In [11]:
def add_attributes(d,p):
    
    f = open(p)
    data = [json.loads(x) for x in f if "EVENT" in x]

    info = []
    for x in data:
        try:
            action = x['datum']['com.bbn.tc.schema.avro.cdm18.Event']['type']
        except:
            action = ''
        try:
            actor = x['datum']['com.bbn.tc.schema.avro.cdm18.Event']['subject']['com.bbn.tc.schema.avro.cdm18.UUID']
        except:
            actor = ''
        try:
            obj = x['datum']['com.bbn.tc.schema.avro.cdm18.Event']['predicateObject']['com.bbn.tc.schema.avro.cdm18.UUID']
        except:
            obj = ''
        try:
            timestamp = x['datum']['com.bbn.tc.schema.avro.cdm18.Event']['timestampNanos']
        except:
            timestamp = ''
        try:
            cmd = x['datum']['com.bbn.tc.schema.avro.cdm18.Event']['properties']['map']['exec']
        except:
            cmd = ''
        try:
            path = x['datum']['com.bbn.tc.schema.avro.cdm18.Event']['predicateObjectPath']['string']
        except:
            path = ''
        try:
            path2 = x['datum']['com.bbn.tc.schema.avro.cdm18.Event']['predicateObject2Path']['string']
        except:
            path2 = ''
        try:
            obj2 = x['datum']['com.bbn.tc.schema.avro.cdm18.Event']['predicateObject2']['com.bbn.tc.schema.avro.cdm18.UUID']
            info.append({'actorID':actor,'objectID':obj2,'action':action,'timestamp':timestamp,'exec':cmd, 'path':path2})
        except:
            pass

        info.append({'actorID':actor,'objectID':obj,'action':action,'timestamp':timestamp,'exec':cmd, 'path':path})

    rdf = pd.DataFrame.from_records(info).astype(str)
    d = d.astype(str)

    return d.merge(rdf,how='inner',on=['actorID','objectID','action','timestamp']).drop_duplicates()

In [9]:
import pandas as pd
import json
f = open("cadets_test.txt")
data = f.read().split('\n')
data = [line.split('\t') for line in data]
df = pd.DataFrame (data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
df = df.dropna()
df.sort_values(by='timestamp', ascending=True,inplace=True)
df = add_attributes(df,"/home/mhoque2/akidul_projects/FLASH/Flash-IDS/testing/ta1-cadets-e3-official-2.json")

In [10]:
phrases, labels, edges, mapp = prepare_graph(df)

In [12]:
import pickle

# Run once after: phrases, labels, edges, mapp = prepare_graph(df)
graph_data = {
    "phrases": phrases,
    "labels": labels,
    "edges": edges,
    "mapp": mapp
}

with open("cadet_test_graphs.pkl", "wb") as f:
    pickle.dump(graph_data, f)

print("✅ Saved graph structure to graph_data.pkl")

✅ Saved graph structure to graph_data.pkl


In [1]:
!pwd

/home/mhoque2/akidul_projects/FLASH/Flash-IDS/testing/hyper_test_eval


#### loading from saved pkl

In [13]:
import pickle

# Load once
with open("cadet_test_graphs.pkl", "rb") as f:
    graph_data = pickle.load(f)

# Extract variables
phrases = graph_data["phrases"]
labels = graph_data["labels"]
edges = graph_data["edges"]
mapp = graph_data["mapp"]

print("✅ Loaded phrases, labels, edges, and mapp from cadet_test_graphs.pkl")


✅ Loaded phrases, labels, edges, and mapp from cadet_test_graphs.pkl


In [9]:
import torch
import numpy as np
import pickle
from collections import defaultdict

# 1. Save node features
x = torch.tensor(nodes, dtype=torch.float)
np.save("cadets_test_features.npy", x.cpu().numpy())          # for DPHGNN
torch.save(x, "cadets_test_x.pt")                             # optional: for PyTorch

# 2. Save labels
y = torch.tensor(labels, dtype=torch.long)
np.save("cadets_test_labels.npy", y.cpu().numpy())
torch.save(y, "cadets_test_y.pt")

# 3. Save edge_index (for baseline GCNs, optional)
edge_index = torch.tensor(edges, dtype=torch.long)
torch.save(edge_index, "cadets_test_edge_index.pt")

# 4. Save UUID → index map
with open("cadets_test_index_map.pkl", "wb") as f:
    pickle.dump(mapp, f)

# 5. Build and save hyperedges based on shared exec/path token
exec_groups = defaultdict(list)

for i, tokens in enumerate(phrases):
    for t in tokens:
        if 'bin/' in t or t.startswith('/'):  # semantic indicator
            exec_groups[t].append(i)

# Keep hyperedges with at least 3 nodes
hyperedges = [group for group in exec_groups.values() if len(group) >= 3]

# Save hyperedge list
with open("cadets_test_edge_list.pkl", "wb") as f:
    pickle.dump(hyperedges, f)

print("✅ Saved: cadets_features.npy, cadets_labels.npy, cadets_edge_list.pkl, cadets_index_map.pkl, and PyTorch versions.")


NameError: name 'nodes' is not defined

In [14]:
for i, node_id in list(enumerate(mapp))[:10]:
    print(f"HGNN index {i} → Original node ID {node_id}")


HGNN index 0 → Original node ID C63B54D6-3DC7-11E8-A5CB-3FA3753A265A
HGNN index 1 → Original node ID C0E60490-62BE-7B5A-BE62-6F2DBA7BA087
HGNN index 2 → Original node ID 0295A18C-3DC8-11E8-A5CB-3FA3753A265A
HGNN index 3 → Original node ID 66BF9E3E-E780-9C5B-80E7-EE1A0B9C8A86
HGNN index 4 → Original node ID 51110606-D232-9652-B2D2-3A7242968F8A
HGNN index 5 → Original node ID 0A5174D8-80FB-4258-BB80-C79A98427D6A
HGNN index 6 → Original node ID 9B091956-B243-9D58-83B2-D0F2D89DB317
HGNN index 7 → Original node ID DE7FB6B3-3DC7-11E8-A5CB-3FA3753A265A
HGNN index 8 → Original node ID 01FB76C2-3DC8-11E8-A5CB-3FA3753A265A
HGNN index 9 → Original node ID 7A350F75-9945-425D-8599-15CEBD426F06


In [14]:
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [15]:
with open("cadets_test_knn_10hop.pkl", "rb") as f:
    edge_list_test = pickle.load(f)
#edge_list_test = edge_list_test.t().tolist() 
#edge_list_test = edge_list_test.tolist() 
# print(type(edge_list_test))
# print(edge_list_test[:5])  # If it's a list


#### new test with attention

In [16]:
import pickle, numpy as np, torch
import torch.nn.functional as F
from dhg import Hypergraph
from dhg.metrics import HypergraphVertexClassificationEvaluator as Evaluator

# ---- import your attention model + helpers from wherever you defined them ----
# from hgnn_attention import AttnHGNN_DHG_pt   # if you put the class in a module
# (assuming AttnHGNN_DHG_pt is already in scope)

def set_seed(s=2023):
    import random
    random.seed(s); np.random.seed(s); torch.manual_seed(s)

if __name__ == "__main__":
    set_seed(2023)

    # If you used the pure-PyTorch (Option A) implementation during training,
    # stick to CPU to avoid the CUDA mismatch
    device = torch.device("cpu")
    print(f"Using device: {device}")

    # --- Load test data
    X_test = torch.tensor(np.load("cadets_test_features.npy"), dtype=torch.float32)
    y_test = torch.tensor(np.load("cadets_test_labels.npy"), dtype=torch.long)

    # Choose the edge list you want for testing and load it
    with open("cadets_test_edge_list_knn_dedup.pkl", "rb") as f:   # <-- pick one
        edge_list_test = pickle.load(f)

    # DHG hypergraph stays as before (Evaluator expects it)
    G_test = Hypergraph(X_test.shape[0], edge_list_test)

    # --- Move to device
    X_test, y_test = X_test.to(device), y_test.to(device)
    G_test = G_test.to(device)

    # --- Build the SAME model class/shape you trained
    num_classes = int(torch.unique(y_test).numel())
    # net = AttnHGNN_DHG_pt(
    #     in_dim=X_test.shape[1],
    #     hid=64,
    #     num_classes=num_classes,
    #     edge_list=edge_list_test,   # <-- pass edge list directly
    #     dropout=0.2,
    #     attn_dim=None,
    #     topk=None
    # ).to(device)

    # # --- Load weights you saved after attention training
    # # (use the actual filename you saved; below uses your latest name)
    # ckpt = torch.load("knn_10_hop_train_cadet_sep_2.pth", map_location=device)
    # net.load_state_dict(ckpt)
    # net.eval()
    net = AttnHGNN_DHG_pt(
    in_dim=X_test.shape[1],
    hid=64,
    num_classes=int(torch.unique(y_test).numel()),
    edge_list=edge_list_test,   # IMPORTANT: test edge list here
    dropout=0.2,
    attn_dim=None,
    topk=None
     ).to(device)

    # --- Load checkpoint, but drop graph-dependent buffers
    ckpt = torch.load("knn_10_hop_train_cadet_sep_2.pth", map_location=device)
    for k in ["V2E", "E2V", "edge_deg"]:
        if k in ckpt: del ckpt[k]

    # Load remaining (weights), letting missing keys slide
    missing = net.load_state_dict(ckpt, strict=False)
    print("Loaded (ignoring graph buffers). Missing/unexpected keys:", missing)

    net.eval()


    # --- Inference & evaluation
    evaluator = Evaluator(["accuracy", "f1_score", {"f1_score": {"average": "micro"}}])
    with torch.no_grad():
        logits = net(X_test, G_test)
        result = evaluator.test(y_test, logits)
        print("Evaluation result on test set:", result)

    # --- Optional: get attention weights, e.g., top-α node per hyperedge (block 1)
    if getattr(net, "_last_alpha", None) is not None:
        a1, a2 = net._last_alpha
        print("Captured attention weights for attribution (block1, block2):", a1.shape, a2.shape)


Using device: cpu
Loaded (ignoring graph buffers). Missing/unexpected keys: _IncompatibleKeys(missing_keys=['V2E', 'E2V', 'edge_deg'], unexpected_keys=[])
Evaluation result on test set: {'accuracy': 0.7600133419036865, 'f1_score': 0.3356861185349467, 'f1_score -> average@micro': 0.7600133268378997}
Captured attention weights for attribution (block1, block2): torch.Size([1406072]) torch.Size([1406072])


In [19]:
with torch.no_grad():
    logits = net(X_test, G_test)
    result = evaluator.test(y_test, logits)
    print("Evaluation result on test set:", result)

    # --- Confidence-based anomaly detection (NEW)
    probs = F.softmax(logits, dim=1)              # class probabilities
    conf, pred = torch.max(probs, dim=1)          # max confidence + predicted class
    threshold = 0.31                               # you can tune this
    low_conf_nodes = (conf < threshold).nonzero(as_tuple=True)[0]

    print(f"\n⚠️ Nodes flagged as suspicious (confidence < {threshold}): {len(low_conf_nodes)}")
    if len(low_conf_nodes) > 0:
        print("Example suspicious node IDs:", low_conf_nodes[:20].tolist())

Evaluation result on test set: {'accuracy': 0.7600133419036865, 'f1_score': 0.3356861185349467, 'f1_score -> average@micro': 0.7600133268378997}

⚠️ Nodes flagged as suspicious (confidence < 0.31): 1334
Example suspicious node IDs: [11, 17, 18, 47, 50, 168, 172, 197, 271, 272, 285, 334, 416, 683, 988, 1054, 1074, 1075, 1087, 1275]


#### Old HGNN

In [27]:
if __name__ == "__main__":
    # --- Setup
    set_seed(2023)
    device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
    #device = torch.device("cpu")

    print(f"Using device: {device}")

    # --- Load test data
    X_test = torch.tensor(np.load("cadets_test_features.npy"), dtype=torch.float)
    y_test = torch.tensor(np.load("cadets_test_labels.npy"), dtype=torch.long)
    # with open("cadets_test_edge_list.pkl", "rb") as f:  #knn
    #     edge_list_test = pickle.load(f)
    # with open("cadets_test_edge_list_knn_dedup.pkl", "rb") as f:  #knn
    #     edge_list_test = pickle.load(f)
    
    # with open("cadets_test_edge_list_hdbscan_filtered.pkl", "rb") as f:    #hdbscan
    #     edge_list_test = pickle.load(f)

    G_test = Hypergraph(X_test.shape[0], edge_list_test)

    # --- Move data to device
    X_test, y_test = X_test.to(device), y_test.to(device)
    G_test = G_test.to(device)

    # --- Load trained model
    net = HGNN(X_test.shape[1], 64, torch.unique(y_test).numel(), use_bn=True).to(device)
    net.load_state_dict(torch.load("knn_10_hop_train_cadet_sep_1.pth"))
    #net.load_state_dict(torch.load("hgnn_cadets_full_training.pth"))   ##knn model testing
    #net.load_state_dict(torch.load("hgnn_cadets_full_training_HDBSCAN.pth")) ##HDBSCSN model testing
    net.eval()

    # --- Inference & Evaluation
    evaluator = Evaluator(["accuracy", "f1_score", {"f1_score": {"average": "micro"}}])
    with torch.no_grad():
        out = net(X_test, G_test)
        result = evaluator.test(y_test, out)
        print(" Evaluation result on test set:", result)


Using device: cuda:3


/tmp/ipykernel_800395/1788347687.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("knn_10_hop_train_cadet_sep_1.pth"))


 Evaluation result on test set: {'accuracy': 0.9892237186431885, 'f1_score': 0.7926043219884891, 'f1_score -> average@micro': 0.9892237396898991}


In [20]:
pred = out.argmax(dim=1)
num_classes = torch.unique(y_test).numel()

tp_per_class = []
fp_per_class = []
fn_per_class = []
tn_per_class = []

for c in range(num_classes):
    TP = ((pred == c) & (y_test == c)).sum().item()
    FP = ((pred == c) & (y_test != c)).sum().item()
    FN = ((pred != c) & (y_test == c)).sum().item()
    TN = ((pred != c) & (y_test != c)).sum().item()
    
    tp_per_class.append(TP)
    fp_per_class.append(FP)
    fn_per_class.append(FN)
    tn_per_class.append(TN)

    print(f"\nClass {c}:")
    print(f"  TP: {TP}")
    print(f"  FP: {FP}")
    print(f"  FN: {FN}")
    print(f"  TN: {TN}")


NameError: name 'out' is not defined

#### Flash Json

In [26]:
import json
with open("cadets_GT.json", "r") as json_file:
    GT_mal = set(json.load(json_file))

#### Orthrus Json

In [31]:
import json
with open("combined_nodes.json", "r") as json_file:
    GT_mal = set(json.load(json_file))

In [27]:
# with torch.no_grad():
#     out = net(X_test, G_test)
#     pred = out.argmax(dim=1)
pred = out.argmax(dim=1)

NameError: name 'out' is not defined

In [18]:
# Step 2: Get misclassified node indices
misclassified_nodes = (pred != y_test).nonzero(as_tuple=False).squeeze()


In [19]:
# Step 3: Map misclassified indices to real node UUIDs
predicted_malicious = set(mapp[i.item()] for i in misclassified_nodes)

In [20]:
with open("cadets_GT.json", "r") as f:
    ground_truth_malicious = set(json.load(f))

In [21]:
# Step 5: Map all node indices in test set to their UUIDs
all_nodes = set(mapp[i] for i in range(len(y_test)))

In [16]:
pred = out.argmax(dim=1)

# Get UUIDs for all predicted malicious = misclassified
misclassified = set(mapp[i.item()] for i in (pred != y_test).nonzero().squeeze())

# Also get UUIDs for correctly classified nodes that are known to be malicious
correct_and_malicious = set(
    mapp[i.item()] for i in (pred == y_test).nonzero().squeeze()
    if mapp[i.item()] in ground_truth_malicious
)

#predicted_malicious = misclassified | correct_and_malicious  # expand prediction scope
predicted_malicious = misclassified 

TP = len(predicted_malicious & ground_truth_malicious)
FP = len(predicted_malicious - ground_truth_malicious)
FN = len(ground_truth_malicious - predicted_malicious)
TN = len(all_nodes - (ground_truth_malicious | predicted_malicious))


NameError: name 'ground_truth_malicious' is not defined

In [23]:
print(f"  TP: {TP}")
print(f"  FP: {FP}")
print(f"  FN: {FN}")
print(f"  TN: {TN}")


  TP: 51
  FP: 3798
  FN: 12807
  TN: 340524


In [37]:
# Optional: Precision, Recall, F1
precision = TP / (TP + FP) if TP + FP > 0 else 0
recall    = TP / (TP + FN) if TP + FN > 0 else 0

f1        = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

print("\nMetrics:")
print(f"  Precision: {precision:.4f}")
print(f"  Recall:    {recall:.4f}")
print(f"  F1 Score:  {f1:.4f}")


Metrics:
  Precision: 0.0133
  Recall:    0.0040
  F1 Score:  0.0061


In [38]:
# Assuming TP, FP, FN, TN are already defined
total = TP + FP + FN + TN
accuracy = (TP + TN) / total if total > 0 else 0

print(f"\nAccuracy: {accuracy:.4f}")



Accuracy: 0.9535


In [28]:
pred = out.argmax(dim=1)

In [63]:
misclassified = set(mapp[i.item()] for i in (pred != y_test).nonzero().squeeze())
print(f"Number of misclassified nodes (treated as malicious): {len(misclassified)}")


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:3 and cpu!

In [34]:
TP = len(misclassified & ground_truth_malicious)

In [35]:
print(TP)

51


In [20]:
def Get_Adjacent(ids, mapp, edges, hops):
    if hops == 0:
        return set()
    
    neighbors = set()
    for edge in zip(edges[0], edges[1]):
        if any(mapp[node] in ids for node in edge):
            neighbors.update(mapp[node] for node in edge)

    if hops > 1:
        neighbors = neighbors.union(Get_Adjacent(neighbors, mapp, edges, hops - 1))
    
    return neighbors

def calculate_metrics(TP, FP, FN, TN):
    FPR = FP / (FP + TN) if FP + TN > 0 else 0
    TPR = TP / (TP + FN) if TP + FN > 0 else 0

    prec = TP / (TP + FP) if TP + FP > 0 else 0
    rec = TP / (TP + FN) if TP + FN > 0 else 0
    fscore = (2 * prec * rec) / (prec + rec) if prec + rec > 0 else 0

    return prec, rec, fscore, FPR, TPR
def helper(MP, all_pids, GP, edges, mapp):
    # --- Strict FLASH logic (before 2-hop)
    TP_raw = MP.intersection(GP)
    FP_raw = MP - GP
    FN_raw = GP - MP
    TN_raw = all_pids - (GP | MP)

    prec_raw, rec_raw, fscore_raw, FPR_raw, TPR_raw = calculate_metrics(
        len(TP_raw), len(FP_raw), len(FN_raw), len(TN_raw)
    )

    print("📊 BEFORE 2-HOP RELAXATION")
    print(f"  TP: {len(TP_raw)}, FP: {len(FP_raw)}, FN: {len(FN_raw)}, TN: {len(TN_raw)}")
    print(f"  Precision: {round(prec_raw, 4)}, Recall: {round(rec_raw, 4)}, F1: {round(fscore_raw, 4)}")
    print(f"  FPR: {round(FPR_raw, 4)}, TPR: {round(TPR_raw, 4)}")

    # --- Apply 2-hop relaxed logic
    two_hop_gp = Get_Adjacent(GP, mapp, edges, 2)
    two_hop_tp = Get_Adjacent(TP_raw, mapp, edges, 2)
    FPL = FP_raw - two_hop_gp
    TPL = TP_raw.union(FN_raw.intersection(two_hop_tp))
    FN = FN_raw - two_hop_tp
    TP, FP, TN = TPL, FPL, TN_raw  # relaxed sets

    prec, rec, fscore, FPR, TPR = calculate_metrics(len(TP), len(FP), len(FN), len(TN))

    print("\n📊 AFTER 2-HOP RELAXATION")
    print(f"  TP: {len(TP)}, FP: {len(FP)}, FN: {len(FN)}, TN: {len(TN)}")
    print(f"  Precision: {round(prec, 4)}, Recall: {round(rec, 4)}, F1: {round(fscore, 4)}")
    print(f"  FPR: {round(FPR, 4)}, TPR: {round(TPR, 4)}")

    return TPL, FPL


#### knn - 10 hop regular - 

In [25]:
# Step 1: Get predicted class labels
pred = out.argmax(dim=1)

# Step 2: Misclassified node indices
misclassified = set(mapp[i.item()] for i in (pred != y_test).nonzero().squeeze())

# Step 3: Load ground truth malicious UUIDs
with open("cadets_GT.json", "r") as f:
    ground_truth_malicious = set(json.load(f))

# Step 4: Create set of all node UUIDs in the test graph
all_node_ids = set(mapp[i] for i in range(len(y_test)))

# Step 5: Run relaxed FLASH evaluation
TPL, FPL = helper(
    MP=misclassified,
    all_pids=all_node_ids,
    GP=ground_truth_malicious,
    edges=edges,  # should be [2, num_edges] format
    mapp=mapp
)

NameError: name 'out' is not defined

#### same approach but evaluated in combined csv/json

In [32]:
# Step 1: Get predicted class labels
pred = out.argmax(dim=1)

# Step 2: Misclassified node indices
misclassified = set(mapp[i.item()] for i in (pred != y_test).nonzero().squeeze())

# Step 3: Load ground truth malicious UUIDs
with open("combined_nodes.json", "r") as f:
    ground_truth_malicious = set(json.load(f))

# Step 4: Create set of all node UUIDs in the test graph
all_node_ids = set(mapp[i] for i in range(len(y_test)))

# Step 5: Run relaxed FLASH evaluation
TPL, FPL = helper(
    MP=misclassified,
    all_pids=all_node_ids,
    GP=ground_truth_malicious,
    edges=edges,  # should be [2, num_edges] format
    mapp=mapp
)

📊 BEFORE 2-HOP RELAXATION
  TP: 36, FP: 3813, FN: 36, TN: 353315
  Precision: 0.0094, Recall: 0.5, F1: 0.0184
  FPR: 0.0107, TPR: 0.5

📊 AFTER 2-HOP RELAXATION
  TP: 46, FP: 282, FN: 26, TN: 353315
  Precision: 0.1402, Recall: 0.6389, F1: 0.23
  FPR: 0.0008, TPR: 0.6389


#### Attention

In [112]:
if torch.is_tensor(mapp):
    mapp = mapp.detach().cpu().tolist()

low_conf_nodes = set(mapp[i.item()] for i in low_conf_nodes.detach().cpu())

AttributeError: 'set' object has no attribute 'detach'

In [85]:
#pred = set(mapp[i.item()] for i in low_conf_nodes.detach().cpu())

#### attention matrixevaluation

In [59]:
# # Step 1: Get predicted class labels
pred = logits.argmax(dim=1)
pred = pred.to(y_test.device)
# MP = low_conf_nodes
#low_conf_nodes = set(mapp[i.item()] for i in low_conf_nodes.detach().cpu())
# # Step 2: Misclassified node indices
misclassified = set(mapp[i.item()] for i in (pred != y_test).nonzero().squeeze())
with open("combined_nodes.json", "r") as f:
    ground_truth_malicious = set(json.load(f))
# Step 4: Create set of all node UUIDs in the test graph
all_node_ids = set(mapp[i] for i in range(len(y_test)))

# Step 5: Run relaxed FLASH evaluation
TPL, FPL = helper(
    MP=low_conf_nodes,
    all_pids=all_node_ids,
    GP=ground_truth_malicious,
    edges=edges,  # should be [2, num_edges] format
    mapp=mapp
)

📊 BEFORE 2-HOP RELAXATION
  TP: 0, FP: 1334, FN: 72, TN: 355794
  Precision: 0.0, Recall: 0.0, F1: 0
  FPR: 0.0037, TPR: 0.0

📊 AFTER 2-HOP RELAXATION
  TP: 0, FP: 0, FN: 72, TN: 355794
  Precision: 0, Recall: 0.0, F1: 0
  FPR: 0.0, TPR: 0.0


### mis plus low conf

In [71]:
with torch.no_grad():
    logits = net(X_test, G_test)
    result = evaluator.test(y_test, logits)
    print("Evaluation result on test set:", result)

    # --- Confidence-based anomaly detection (NEW)
    probs = F.softmax(logits, dim=1)              # class probabilities
    conf, pred = torch.max(probs, dim=1)          # max confidence + predicted class
    threshold = 0.311                               # you can tune this
    low_conf_nodes = (conf < threshold).nonzero(as_tuple=True)[0]

    print(f"\n⚠️ Nodes flagged as suspicious (confidence < {threshold}): {len(low_conf_nodes)}")
    if len(low_conf_nodes) > 0:
        print("Example suspicious node IDs:", low_conf_nodes[:20].tolist())

Evaluation result on test set: {'accuracy': 0.7600133419036865, 'f1_score': 0.3356861185349467, 'f1_score -> average@micro': 0.7600133268378997}

⚠️ Nodes flagged as suspicious (confidence < 0.311): 1364
Example suspicious node IDs: [11, 17, 18, 47, 50, 168, 172, 197, 271, 272, 285, 334, 416, 683, 988, 1054, 1074, 1075, 1087, 1275]


In [72]:
if torch.is_tensor(mapp):
    mapp = mapp.detach().cpu().tolist()
low_conf_nodes = set(mapp[i.item()] for i in low_conf_nodes.detach().cpu())

In [46]:


# # Step 1: Get predicted class labels
# pred = logits.argmax(dim=1)
# pred = pred.to(y_test.device)
# MP = low_conf_nodes
#low_conf_nodes = set(mapp[i.item()] for i in low_conf_nodes.detach().cpu())
# # Step 2: Misclassified node indices
misclassified = set(mapp[i.item()] for i in (pred != y_test).nonzero().squeeze())
MP = misclassified & low_conf_nodes

with open("cadets_GT.json", "r") as f:
    ground_truth_malicious = set(json.load(f))
# with open("combined_nodes.json", "r") as f:
#     ground_truth_malicious = set(json.load(f))
# Step 4: Create set of all node UUIDs in the test graph
all_node_ids = set(mapp[i] for i in range(len(y_test)))

# Step 5: Run relaxed FLASH evaluation
TPL, FPL = helper(
    MP=MP,
    all_pids=all_node_ids,
    GP=ground_truth_malicious,
    edges=edges,  # should be [2, num_edges] format
    mapp=mapp
)

📊 BEFORE 2-HOP RELAXATION
  TP: 0, FP: 387, FN: 12858, TN: 343935
  Precision: 0.0, Recall: 0.0, F1: 0
  FPR: 0.0011, TPR: 0.0

📊 AFTER 2-HOP RELAXATION
  TP: 0, FP: 0, FN: 12858, TN: 343935
  Precision: 0, Recall: 0.0, F1: 0
  FPR: 0.0, TPR: 0.0


#### reaper eval

In [73]:


# # Step 1: Get predicted class labels
pred = logits.argmax(dim=1)
pred = pred.to(y_test.device)
# MP = low_conf_nodes
#low_conf_nodes = set(mapp[i.item()] for i in low_conf_nodes.detach().cpu())
# # Step 2: Misclassified node indices
misclassified = set(mapp[i.item()] for i in (pred != y_test).nonzero().squeeze())
MP = misclassified & low_conf_nodes

with open("/home/mhoque2/akidul_projects/FLASH/Flash-IDS/testing/hyper_test_eval/reaper/", "r") as f:
    ground_truth_malicious = set(json.load(f))
# with open("combined_nodes.json", "r") as f:
#     ground_truth_malicious = set(json.load(f))
# Step 4: Create set of all node UUIDs in the test graph
all_node_ids = set(mapp[i] for i in range(len(y_test)))

# Step 5: Run relaxed FLASH evaluation
TPL, FPL = helper(
    MP=MP,
    all_pids=all_node_ids,
    GP=ground_truth_malicious,
    edges=edges,  # should be [2, num_edges] format
    mapp=mapp
)

📊 BEFORE 2-HOP RELAXATION
  TP: 3, FP: 1144, FN: 1425, TN: 355397
  Precision: 0.0026, Recall: 0.0021, F1: 0.0023
  FPR: 0.0032, TPR: 0.0021

📊 AFTER 2-HOP RELAXATION
  TP: 632, FP: 0, FN: 796, TN: 355397
  Precision: 1.0, Recall: 0.4426, F1: 0.6136
  FPR: 0.0, TPR: 0.4426


In [ ]:
low_conf_nodes =low_conf_nodes.to(y_test.device)

In [42]:
print("🔹 Misclassified nodes (MP):", len(misclassified))
print("🔹 Ground truth malicious nodes (GP):", len(ground_truth_malicious))
print("🔹 All test node UUIDs:", len(mapp))


🔹 Misclassified nodes (MP): 3841
🔹 Ground truth malicious nodes (GP): 12858
🔹 All test node UUIDs: 357174


In [43]:
print("\n🧾 Sample Misclassified:", list(misclassified)[:5])
print("🧾 Sample Ground Truth Malicious:", list(ground_truth_malicious)[:5])
print("🧾 Sample TP after 2-hop (TPL):", list(TPL)[:5])
print("🧾 Sample FN after 2-hop:", list((ground_truth_malicious - TPL))[:5])



🧾 Sample Misclassified: ['AE3F28A0-3E57-11E8-A5CB-3FA3753A265A', 'CBEE30D0-D58B-5DE5-A30C-74DAA6FBE323', '5B7B71CA-1921-975F-A119-1DAC3F972861', '854EEED7-3DE6-11E8-A5CB-3FA3753A265A', '72C11964-85F8-53DA-95A7-ABE856B4DEB1']
🧾 Sample Ground Truth Malicious: ['797BB893-3E80-11E8-A5CB-3FA3753A265A', '71AAAE25-3E80-11E8-A5CB-3FA3753A265A', 'A05C089A-3E80-11E8-A5CB-3FA3753A265A', '9AF29D7C-3E80-11E8-A5CB-3FA3753A265A', '9EE588D2-3E80-11E8-A5CB-3FA3753A265A']
🧾 Sample TP after 2-hop (TPL): ['797BB893-3E80-11E8-A5CB-3FA3753A265A', '71AAAE25-3E80-11E8-A5CB-3FA3753A265A', 'A05C089A-3E80-11E8-A5CB-3FA3753A265A', '9AF29D7C-3E80-11E8-A5CB-3FA3753A265A', '9EE588D2-3E80-11E8-A5CB-3FA3753A265A']
🧾 Sample FN after 2-hop: ['2A03F3BA-792D-9C5B-AD79-5A5DEB9CB9D2', '0C773AFD-8F2A-3555-AA8F-AFF835357207', '4FB0BFEA-3F1C-11E8-A5CB-3FA3753A265A', 'F5B43D0A-C3F6-AE54-B6C3-B6F0B4AE4591', '885AF67E-0E96-FE5D-960E-14BA5DFEBBD3']


In [44]:
    two_hop_gp = Get_Adjacent(GP, mapp, edges, 2)
    two_hop_tp = Get_Adjacent(TP, mapp, edges, 2)

NameError: name 'GP' is not defined

In [22]:
def helper(MP, all_pids, GP, edges, mapp):
    # --- Strict FLASH logic (before 2-hop)
    TP_raw = MP.intersection(GP)
    FP_raw = MP - GP
    FN_raw = GP - MP
    TN_raw = all_pids - (GP | MP)

    prec_raw, rec_raw, fscore_raw, FPR_raw, TPR_raw = calculate_metrics(
        len(TP_raw), len(FP_raw), len(FN_raw), len(TN_raw)
    )

    print("📊 BEFORE 2-HOP RELAXATION")
    print(f"  TP: {len(TP_raw)}, FP: {len(FP_raw)}, FN: {len(FN_raw)}, TN: {len(TN_raw)}")
    print(f"  Precision: {round(prec_raw, 4)}, Recall: {round(rec_raw, 4)}, F1: {round(fscore_raw, 4)}")
    print(f"  FPR: {round(FPR_raw, 4)}, TPR: {round(TPR_raw, 4)}")

    # --- Apply 2-hop relaxed logic
    two_hop_gp = Get_Adjacent(GP, mapp, edges, 3)
    two_hop_tp = Get_Adjacent(TP_raw, mapp, edges, 3)
    FPL = FP_raw - two_hop_gp
    TPL = TP_raw.union(FN_raw.intersection(two_hop_tp))
    FN = FN_raw - two_hop_tp
    TP, FP, TN = TPL, FPL, TN_raw  # relaxed sets

    prec, rec, fscore, FPR, TPR = calculate_metrics(len(TP), len(FP), len(FN), len(TN))

    print("\n📊 AFTER 2-HOP RELAXATION")
    print(f"  TP: {len(TP)}, FP: {len(FP)}, FN: {len(FN)}, TN: {len(TN)}")
    print(f"  Precision: {round(prec, 4)}, Recall: {round(rec, 4)}, F1: {round(fscore, 4)}")
    print(f"  FPR: {round(FPR, 4)}, TPR: {round(TPR, 4)}")

    return TPL, FPL


In [23]:
!pwd

/home/mhoque2/akidul_projects/FLASH/Flash-IDS/testing/hyper_test_eval


In [24]:
TPL, FPL = helper(
    MP=misclassified,
    all_pids=all_node_ids,
    GP=ground_truth_malicious,
    edges=edges,  # should be [2, num_edges] format
    mapp=mapp
)

NameError: name 'all_node_ids' is not defined